In [17]:
import requests

response = requests.get("http://127.0.0.1:8000/get_template_list")
if response.status_code == 200:
    print(response.json())
else:
    print(f"Error: {response.status_code}, {response.text}")

['Generator London', "Hazlitt's", 'Muji Hotel Ginza']


In [19]:
response = requests.get("http://127.0.0.1:8000/get_template", params={"template_name": "Hazlitt's"})
print(response.json())

{'Hotel Name': "Hazlitt's", 'Location': "Soho, London, United Kingdom (in the heart of the West End’s Theatreland) (Hotel Hazlitt's, London, United Kingdom - www.trivago.com).", 'Architectural Style': "Georgian townhouse boutique hotel with interiors that reflect its 18th-century history. Rooms feature carved oak four-poster beds, rich fabrics, and Victorian-era fittings, transporting guests back in time (Hotel Hazlitt's, London, United Kingdom - www.trivago.com).", 'Service Highlights': "Old-world hospitality with modern touches: an elegant library lounge with a cozy fireplace and an honesty bar for evening drinks (Hotel Hazlitt's, London, United Kingdom - www.trivago.com), indulgent in-room breakfasts of fresh pastries and coffee (Hotel Hazlitt's, London, United Kingdom - www.trivago.com), and 24-hour attentive service. Double-paned windows preserve quiet comfort amid the lively Soho streets, ensuring a peaceful night’s sleep (Hotel Hazlitt's, London, United Kingdom - www.trivago.com

In [8]:
id_dict = requests.get("http://127.0.0.1:8000/presets").json()
print(id_dict)

[{'id': 'official_website', 'name': 'Official Website Style'}, {'id': 'professional', 'name': 'Professional Tone'}]


In [39]:
import requests

# Base URL of your API
base_url = "http://localhost:8000"  # Replace with your actual server URL

# Make the GET request
response = requests.get(f"{base_url}/presets")

# Check the response
if response.status_code == 200:
    print("Preset data:", response.json())
else:
    print("Error:", response.status_code, response.json())

Preset data: {'scenario': {'Website': "You are a professional copywriter writing for a hotel's official website.\nTone: polished, informative, elegant.", 'OTA': 'You are crafting engaging OTA (Online Travel Agency) copy.\nTone: concise, benefit-driven, trust-building.', 'Social': 'You are creating a viral social-media post.\nTone: snappy, friendly, hashtag-ready.'}, 'style': {'japanese_minimal': 'Style: Japanese minimalist—clean, calm, nature‑inspired.', 'english_vintage': 'Style: English vintage—ornate, historic, warm.', 'youth_social': 'Style: Youthful social—fresh, colloquial, emoji‑friendly.'}}


In [42]:
r = response.json()
r['style'].keys()

dict_keys(['japanese_minimal', 'english_vintage', 'youth_social'])

In [13]:
name = requests.get(f"http://127.0.0.1:8000/presets/{id_dict[0]['id']}").json()
print(name)

{'detail': 'Not Found'}


In [1]:
with open("presets/template", "r") as f:
    template = f.read()
    
print(template)

You are a professional travel copywriting assistant to generate hotel descriptions.

Here is the hotel information:
{{ hotel_info }}

You should use "{{ style }}" style to write the description for the scenario of "{{ scenario }}".

Please generate:

1. **A one-line catchy headline**
2. **A concise introduction** (2-3 sentences)
3. **Three bullet-point highlights**

---

### Style Guidance for "{{ style }}":
{{ style_tone }}

### Scenario Instructions for "{{ scenario }}":
{{ scenario_tone }}

---

Please write in English if no language is specified.



In [10]:
import json
import jinja2

# 加载JSON数据
styles = json.load(open("presets/styles.json"))
scenarios = json.load(open("presets/scenarios.json"))

# 选定style与scenario
selected_style = "Japanese Minimal"
selected_scenario = "Website"

# 酒店信息 (示例字典)
hotel_info = {
    "name": "Kyoto Serenity Hotel",
    "location": "Kyoto, Japan",
    "unique_selling_points": ["Zen garden", "Minimalist rooms", "Tea ceremony experience"]
}

def hotel_info2str(hotel_info):
    lines = []
    for key, value in hotel_info.items():
        if isinstance(value, list):
            value = ", ".join(value)
        key = key.replace("_", " ").title()
        lines.append(f"- **{key}**: {value}")
    return "\n".join(lines)

# 渲染prompt
template = jinja2.Template(open("presets/template").read())

prompt_text = template.render(
    hotel_info=hotel_info2str(hotel_info),
    style=selected_style,
    style_tone=styles[selected_style],
    scenario=selected_scenario,
    scenario_tone=scenarios[selected_scenario]
)

# 接下来把 prompt_text 发送给 Ollama 等大模型接口即可
with open("presets/prompt.txt", "w") as f:
    f.write(prompt_text)

In [39]:
import json
import jinja2
from pathlib import Path

def generate_hotel_prompt(selected_style: str, selected_scenario: str, hotel_info: dict, prompt: str = "") -> str:

    # Load presets
    base_path = Path("presets")
    styles = json.load(open(base_path / "styles.json"))
    scenarios = json.load(open(base_path / "scenarios.json"))
    template = jinja2.Template(open(base_path / "template").read())

    # hotel_info -> markdown bullet
    def hotel_info2str(hotel_info):
        lines = []
        for key, value in hotel_info.items():
            if isinstance(value, list):
                value = ", ".join(value)
            key = key.replace("_", " ").title()
            lines.append(f"- **{key}**: {value}")
        return "\n".join(lines)

    # Render final prompt
    rendered = template.render(
        hotel_info=hotel_info2str(hotel_info),
        style=selected_style,
        style_tone=styles[selected_style],
        scenario=selected_scenario,
        scenario_tone=scenarios[selected_scenario],
    )
    
    # Add additional prompt if provided
    if prompt:
        rendered += f"\n\nPlease also consider the following:\n{prompt}"

    return rendered

generated_prompt = generate_hotel_prompt(
    selected_style="Youth Social",
    selected_scenario="Website",
    hotel_info={
        "name": "高定酒店",
        "location": "Kyoto, Japan",
        "unique_selling_points": ["Zen garden", "Minimalist rooms", "Tea ceremony experience"]
    },
    prompt="Please use Chinese to generate text"
)

with open("presets/prompt.txt", "w") as f:
    f.write(generated_prompt)

In [40]:
import ollama

client = ollama.Client("http://localhost:11434")
model = "phi3:3.8b"
model = "qwen2.5:latest"

response = client.generate(model=model, prompt=generated_prompt)
with open("presets/response.md", "w") as f:
    f.write(response.response)

In [49]:
response['response']

"# Title\nZen Out in Kyoto: High-Quality Relaxation at 高定酒店\n\n# Introduction\nExperience pure tranquility at 高定酒店, where minimalist vibes meet ancient Japanese charm. Nestled in the heart of Kyoto, this hotel is more than just a place to stay; it's your gateway to a serene escape with a private Zen garden and an authentic tea ceremony experience.\n\n# Highlights\n- **Zen Garden Retreat**: Step into nature’s embrace with a personal Zen garden right outside your door.\n- **Minimalist Luxe Rooms**: Wake up to sleek, modern spaces that offer both comfort and calm.\n- **Tea Ceremony Traditions**: Immerse yourself in the art of the tea ceremony for a taste of true Japanese culture."